In [1]:
import os

In [2]:
%pwd

'/Users/rupeshyadav/Documents/MLFLOW/End-To-End-MLOPs-Credit_Card_Fraud_Detection/research'

In [15]:
os.chdir("../")

In [16]:
%pwd

'/Users/rupeshyadav/Documents/MLFLOW/End-To-End-MLOPs-Credit_Card_Fraud_Detection'

In [17]:
from dataclasses import dataclass
from pathlib import Path

In [18]:
@dataclass(frozen=True)
class DataIngestionConfig:
    """
    A class for holding an data path
    """
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    
    


In [19]:
from Credit_Card_Fraud_Detection.constants import *
from Credit_Card_Fraud_Detection.utils.common import read_yaml,create_directories

In [20]:
print(CONFIG_FILE_PATH)

config/config.yaml


In [21]:
print(PARAMS_FILE_PATH)

params.yaml


In [22]:
print(SCHEMA_FILE_PATH)

schema.yaml


In [23]:
print("hi")

hi


In [24]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_file_path),
        self.params = read_yaml(params_file_path),
        self.schema = read_yaml(schema_file_path)
    
        print(self.config)
        print(self.config[0].artifacts_root)
        create_directories([self.config[0].artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config[0].data_ingestion
        create_directories([config.root_dir])   
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir 
         )
        
        return data_ingestion_config

 

In [25]:
import os
import urllib.request as request
import zipfile
from Credit_Card_Fraud_Detection import logger
from Credit_Card_Fraud_Detection.utils.common import get_size

In [48]:
class DataIngestionStrategy:
    def __init__(self,config: DataIngestionConfig):
        self.config = config
        print("="*50)
        print(self.config.local_data_file)
        print("="*50)
        
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with the following \n {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  
        


In [50]:
# Pipelines for the DataIngestionStrategy
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestionStrategy(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-01-01 15:29:53,493: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-01 15:29:53,495: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-01 15:29:53,497: INFO: common: yaml file: schema.yaml loaded successfully]
(ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/Rupesh-132/End-To-End-MLOPs-Credit_Card_Fraud_Detection/raw/main/data/credit_card_dataset.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}),)
artifacts
[2024-01-01 15:29:53,499: INFO: common: created directory at: artifacts]
[2024-01-01 15:29:53,500: INFO: common: created directory at: artifacts/data_ingestion]
artifacts/data_ingestion/data.zip
[2024-01-01 15:29:55,788: INFO: 2886160661: artifacts/data_ingestion/data.zip downloaded! with the following 
 Connection: close
Content-Length: 69155672
Cache-Control: max-age=300
Content-Security-Polic